# Punti di interesse

In [ ]:
from shapely import Point

import geopandas as gpd

from my_paths import *

# Municipi

In [ ]:
gdf = gpd.read_file(PATH_MUNICIPI_RAW)
gdf = gdf.to_crs("EPSG:4326")
gdf.to_file(PATH_MUNICIPI_CLEAN)

# Parchi

In [20]:
gdf = gpd.read_file(PATH_PARCHI_RAW)
gdf.to_file(PATH_PARCHI_CLEAN)

# Fontane

Carichiamo Raw csv

In [ ]:
gdf = gpd.read_file(PATH_FONTANE_RAW, sep=";")
gdf["Location"] = gdf["Location"].apply(lambda x: Point((float(x.strip("(").strip(")").strip().split(",")[1]),
                                                        float(x.strip("(").strip(")").strip().split(",")[0]))))
gdf = gdf.rename(columns = {'objectID':'id_fontane',
                           'MUNICIPIO':'municipio',
                           'ID_NIL': 'id_nome',
                           'NIL': "nome",
                           'LONG_X_4326':'longitudine',
                           'LAT_Y_4326': 'latitudine',
                           'Location':'geometry'})
gdf['icon'] = "events"
gdf = gpd.GeoDataFrame(gdf, geometry="geometry", crs="EPSG:4326")
gdf.to_file(PATH_FONTANE_CLEAN, driver="GeoJSON")

# Impianti sportivi

1. Rimozione righe in cui non è segnalata una data inizio costruzione
2. Rimozione delle piste ciclabili, dato che abbiamo un dataset apposito
3. Convertiamo la data in un intero, in modo da poter filtrare agilmente su kepler

In [ ]:
gdf = gpd.read_file(PATH_IMPIANTI_SPORTIVI_RAW)
gdf = gdf[["municipio", "località", "obj_id", "data_ini", "area_mq", "perim_m", "descrizione_codice", "geometry"]]
gdf = gdf.rename(columns={"data_ini": "anno_inizio", "obj_id": "id_impianto"})

gdf = gdf[gdf["anno_inizio"].notna()].reset_index()
gdf["anno_inizio"] = gdf["anno_inizio"].apply(lambda x: x.year)

gdf = gdf[~gdf["descrizione_codice"].str.contains("pista ciclabile", case=False, na=False)].reset_index()
gdf = gdf.drop(columns=["level_0", "index"], errors="ignore")

Salviamo in Clean

In [ ]:
gdf.to_file(PATH_IMPIANTI_SPORTIVI_CLEAN, driver="GeoJSON")

# Biblioteche

In [ ]:
gdf = gpd.read_file(PATH_BIBLIOTECHE_RAW)
print(type(gdf))
column = {
    "Indirizzo": "Indirizzo",
    "CAP": "CAP",
    "MUNICIPIO": "Municipio",
    "NIL": "Quartiere",
    "long": "Longitudine",
    "lat": "Latitudine",
    "geometry": "geometry"
}
gdf = gdf[column.keys()]
gdf["icon"] = "place"
gdf = gdf.rename(columns=column)

gdf.to_file(PATH_BIBLIOTECHE_CLEAN, driver="GeoJSON")

# Scuole

Carichiamo tutti i file raw come gdf

In [ ]:
gdf_primarie = gpd.read_file(PATH_SCUOLE_PRIMARIE_RAW)
gdf_secondarie_1 = gpd.read_file(PATH_SCUOLE_SECONDARIE_PRIMO_RAW)
gdf_secondarie_2 = gpd.read_file(PATH_SCUOLE_SECONDARIE_SECONDO_RAW)
gdf_universita = gpd.read_file(PATH_UNIVERSITA_RAW)

Scuole primarie e secondarie di primo grado possiedono la stessa granularità quindi effettiuamo le stesse operazioni per entrambi.  
1. Filtro solo per ANNOSCOL 2023/2024  
2. Rinomino colonne che ci servono e tengo solo quelle

In [ ]:
column_prim_sec1 = {
    "DENOMINAZIONE": "Denominazione",
    "GRADO": "Grado",
    "INDIRIZZO": "Indirizzo",
    "MUNICIPIO": "Municipio",
    "NIL": "Quartiere",
    "LONG_X_4326": "Longitudine",
    "LAT_Y_4326": "Latitudine",
    "geometry": "geometry"
}
# Modifico nomi e tengo solo colonne che mi serono per primarie
gdf_primarie = gdf_primarie[gdf_primarie["ANNOSCOL"] == 2324].reset_index()
gdf_primarie = gdf_primarie.rename(columns=column_prim_sec1)[column_prim_sec1.values()]
# e secondarie 
gdf_secondarie_1 = gdf_secondarie_1[gdf_secondarie_1["ANNOSCOL"] == 2324].reset_index()
gdf_secondarie_1 = gdf_secondarie_1.rename(columns=column_prim_sec1)[column_prim_sec1.values()]
gdf_secondarie_1["Grado"] = "Scuola secondaria - Primo grado"

Nulla da sistemare, faccio solo il rename delle colonne ed estraggo dal df solo quelle

In [ ]:
column_sec2 = {
    "DENOMINAZ": "Denominazione",
    "TIPOLOGIA": "Grado",
    "INDIRIZZO": "Indirizzo",
    "MUNICIPIO": "Municipio",
    "NIL": "Quartiere",
    "LONG_X_4326": "Longitudine",
    "LAT_Y_4326": "Latitudine",
    "geometry": "geometry"
}
gdf_secondarie_2 = gdf_secondarie_2.rename(columns=column_sec2)[column_sec2.values()]
gdf_secondarie_2["Grado"] = "Scuola secondaria - Secondo grado"

Uguale per università

In [ ]:
column_uni = {
    "DENOMINAZ": "Denominazione",
    "TIPOLOGIA": "Grado",
    "INDIRIZZO": "Indirizzo",
    "MUNICIPIO": "Municipio",
    "NIL": "Quartiere",
    "LONG_X_4326": "Longitudine",
    "LAT_Y_4326": "Latitudine",
    "geometry": "geometry"
}
gdf_universita = gdf_universita.rename(columns=column_uni)[column_uni.values()]
gdf_universita["Grado"] = "Università"

Uniamo i 4 geodataframe ripuliti e portati alla stessa granularità

In [ ]:
final_gdf = gpd.pd.concat([gdf_primarie, gdf_secondarie_1, gdf_secondarie_2, gdf_universita], ignore_index=True)
final_gdf = gpd.GeoDataFrame(final_gdf, geometry=gdf_primarie.geometry.name)

Aggiungo colonna icona per la visualizzazione su kepler

In [ ]:
final_gdf["icon"] = "id-card"

Salviamo il file in Clean

In [ ]:
final_gdf.to_file(PATH_SCUOLE_CLEAN, driver="GeoJSON")